## Functions trash bin

### Modifying file

In [1]:
import json

def get_data(filename='data.json'):
    with open(filename, 'r') as f:
        return json.load(f)


def save_data(data, filename='data.json'):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=2)

### Filtering data

In [64]:
def filter_data(**attributes):
    data = get_data()
    result = dict()

    for k, v in data.items():
        if not any(v[k1] not in v1 for k1, v1 in attributes.items()):
            result[k] = v
    
    return result

### Updating algs

In [60]:
get_sheet_content('algs.csv')

,0,1,2,3
0,UF,UB,UL,UR
1,UB,NaN,"U' M2 U: U2, M",R2 U' S' U2 S U' R2
2,UL,"U' M2 U: M, U2",NaN,"M2 U: M, U2"
3,UR,"R2 U': S, R2","M2 U': M, U2",NaN


In [62]:
df = get_sheet_content('algs.csv')
df_to_alg_list(df)


4
4


["UF;UL;UB;U' M2 U: U2, M",
 "UF;UR;UB;R2 U' S' U2 S U' R2",
 "UF;UB;UL;U' M2 U: M, U2",
 'UF;UR;UL;M2 U: M, U2',
 "UF;UB;UR;R2 U': S, R2",
 "UF;UL;UR;M2 U': M, U2"]

In [61]:
import pandas as pd
import numpy as np

VALID_CHARS = " UDFBRLMESudfbrlw'/:,2xyz"

def get_sheet_content(filename):
    df = pd.read_csv(filename, header=None, sep=';')
    df = df.replace(np.nan,'',regex=True)
    return df


def df_to_alg_list(df):
    result = []
    print(len(df))
    print(len(df.iloc[0]))

    for i in range(1, len(df)):
        for j in range(1, len(df.iloc[0])):
            if df.iloc[i][j]:
                alg = clean_alg_entry(df.iloc[i][j])
                key = ";".join([df.iloc[0][0], df.iloc[0][j], df.iloc[i][0], alg])
                result.append(key)
    return result


def clean_alg_entry(alg):
    # filtering out invalid chars
    alg = ''.join([i for i in alg if i in VALID_CHARS])

    # cleaning multiple spaces 
    alg = ' '.join(alg.split())

    # cleaning whitespaces before , : '
    i = 0
    while i < len(alg):
        if alg[i] in "':," and i > 0 and alg[i - 1] == " ":
            alg = alg[:i-1] + alg[i:]
        else:
            i += 1

    return alg


def only_latest_algs(data):
    return {k: v for k, v in data.items() if v['latest_alg']}


def keys_with_given_buffer_and_targets(data, buffer, first_target, second_target):
    return [k for k in data if [buffer, first_target, second_target] == k.split(';')[:3]]


def is_key_in_dict(data, key):
    return key in data


def new_record_from_key(key):
    key = key.split(';')

    return {
        'buffer': key[0],
        'first_target': key[1],
        'second_target': key[2],
        'alg': key[3],
        'results': [],
        'latest': True
    }


def update_algs():
    data = get_data()
    algs = get_sheet_content('algs.csv')
    algs_list = df_to_alg_list(algs)

    for alg in algs_list:
        decoded_key = alg.split(';')
        existing_algs = keys_with_given_buffer_and_targets(data, decoded_key[0], decoded_key[1], decoded_key[2])

        for k in existing_algs:
            data[k]['latest'] = False

        if alg in data:
            data[alg]['latest'] = True
        else:
            data[alg] = new_record_from_key(alg)

    return data

In [4]:
data = update_algs()
save_data(data, 'data_updated.json') 

### Updating memo

In [7]:
def get_words_dict(grid):
    result = dict()
    for i in range(1, len(grid)):
        for j in range(1, len(grid[0])):
            if grid[i][j]:
                key = f'{grid[0][j]};{grid[i][0]}'
                result[key] = grid[i][j]
    return result


def update_words():
    data = get_data()
    words = get_sheet_content('memo.csv')
    words_dict = get_words_dict(words)

    for k, v in data.items():
        try:
            targets = f"{v['first_target']};{v['second_target']}"
            data[k]['word'] = words_dict[targets]
        except KeyError:
            pass
            
    return data

### Updating LPs

In [9]:
def get_lps_dict(grid):
    return {i[0]: i[1] for i in grid}

def update_LPs():
    data = get_data()
    lps = get_sheet_content('LP.csv')
    lps_dict = get_lps_dict(lps)

    for k, v in data.items():
        try:
            targets = [v['first_target'], v['second_target']]
            data[k]['lp'] = ''.join([lps_dict[i] for i in targets])
        except KeyError:
            pass
    
    return data

### Running game

In [17]:
from random import shuffle
from datetime import datetime

def get_n_random_keys(data, n):
    data = {k: v for k, v in data.items() if v['latest']}
    keys = list(data)
    shuffle(keys)
    return keys[:n]    


def measure_time():
    start = datetime.now()
    input()
    end = datetime.now()
    delta = (end - start).total_seconds()
    return round(delta, 2)


def display_record(alg):
    if 'word' in alg:
        print(alg['word'])
    elif 'lp' in alg:
        print(alg['lp'])
    else:
        print(f"{alg['first_target']} {alg['second_target']}")


def get_response():
    print()
    response = input('Type Y to save results')
    return response == 'Y'


def add_entries(data, results):
    for k, v in results.items():
        data[k]['results'].append(v)

    save_data(data)

def run_game(n):
    results_dict = dict()
    data = get_data()
    algs = get_n_random_keys(data, n)
    for alg in algs:
        display_record(data[alg])
        exec_time = measure_time()
        results_dict[alg] = exec_time

    print('Results to be saved:')
    for k, v in results_dict.items():
        print(' '.join(k.split(';')[1:3]), v)

    if get_response():
        add_entries(data, results_dict)

### El śmietniko

In [66]:
query = {
    "latest": [False]
}

filtered_data = filter_data(**query)
for k, v in filtered_data.items():
    print(v['buffer'], v['first_target'], v['second_target'])
    print(v['alg'])
    print()

UF UR UB
R2 U: S, R2



In [43]:
alg = "[Ra    :  R  (1E ' 6  R    '&&   , %9 U       ']"
clean_alg_entry(alg)

"R: R E' R', U'"

In [68]:
save_data(update_words())

In [69]:
run_game(5)

Arbuz
Bat
Cable
Acid
Baca
Results to be saved:
UB UL 0.71
UL UB 0.92
UR UL 1.29
UB UR 2.81
UL UR 3.06



In [31]:
import pandas as pd

df = pd.read_excel('data.xlsx', sheet_name = None) # can also index sheet by name or fetch all sheets
df_uf = df['UF']

In [35]:
df_uf.iloc[0][0]

'UB'

In [37]:
df_uf.keys()[0]

'UF'

In [44]:
df2 = pd.read_csv('algs.csv', sep=';', header=None)

In [45]:
df2

,0,1,2,3
0,UF,UB,UL,UR
1,UB,NaN,"U' M2 U: U2, M",R2 U' S' U2 S U' R2
2,UL,"U' M2 U: M, U2",NaN,"M2 U: M, U2"
3,UR,"R2 U': S, R2","M2 U': M, U2",NaN


In [47]:
df2.iloc[2][1]

"U' M2 U: M, U2"

TypeError: 'float' object is not iterable